# SMPC - Simple Aggregation Algorithm 

This notebook serves as the initial integration of a simple aggregation algorithm between 3+ parties without revealing their secret values using open source protocols. This is achieved by splitting each value into multiple shares, each of which operate like a private key.

Important notes:
* each secret is split into 3 shares for now.
* honest-but-curious (or passive) security


## Preparations

In [1]:
# import modules
import random as rd  # generating random numbers for distributions
import numpy as np  # handling vectors and matrices
from generate_prime import generate_prime_number  # code for generating large prime numbers quickly

## Encryption

> Encryption doesn't use floats or real numbers but happens in a mathematical space called [integer quotient ring](http://mathworld.wolfram.com/QuotientRing.html) which is basically the integers between `0` and `Q-1`, where `Q` is prime and \"big enough\" so that the space can contain all the numbers that we use in our experiments. In practice, given a value `x` integer, we do `x % Q` to fit in the ring. (That's why we avoid using number `x' > Q`).

In [2]:
Q = generate_prime_number(50)
Q

1103316058601747

In [88]:
def encrypt(x, n_shares=3):
    shares = tuple(rd.randrange(0,Q) for _ in range(n_shares-1))
    return (*shares, ((x - sum(shares)) % Q))

In [89]:
shares = encrypt(6041)
shares

(319041439621850, 961122320409415, 926468357178270)

## Decryption

In [91]:
def decrypt(*shares):
    return sum(shares) % Q

In [92]:
decrypt(*shares)

6041

## Aggregation Function

`x + y = (x0 + x1 + x2) + (y0 + y1 + y2)`

`x + y = (x0 + y0) + (x1 + y1) + (x2 + y2)`

In [123]:
def add(x, y):
    return [(x[i] + y[i]) % Q for i in range(len(x))]  # x & y have to have the same number of shares (= length)

### Example for adding 3 secrets

In [124]:
# example
# set 3 secret values and encrypt each into 3 (default) shares
x = encrypt(6041)
y = encrypt(59)
z = encrypt(900)
# what do the shares look like
[x, y, z]

[(603491997781816, 631845925417737, 971294194009982),
 (702683904294614, 601958935897033, 901989277011906),
 (708402705181507, 533783530249363, 964445881773524)]

In [125]:
# add the encrypted values
agg = add(add(x,y),z)
agg

[2014578607257937, 1767588391564133, 2837729352795412]

In [126]:
# and decrypt the aggregated shares
decrypt(*agg)

7000

## Multiplication Function

`x * y = (x0 + x1 + x2) * (y0 + y1 + y2)`

`x * y = (x0 * y0) + (x1 * y1) + (x2 * y2) + (x0 * y1) + (x0 * y2) + (x1 * y0) + (x1 * y2) + (x2 * y0) + (x2 * y1)`

* masks: s, t
* masked values: alpha, beta

`z0 = st0 + (s0 * beta) + (alpha * t0) + (alpha * beta)`

`z1 = st1 + (s1 * beta) + (alpha * t1)`

`alpha = (x0 - s0) + (x1 - s1)`

`beta = (y0 - t0) + (y1 - t1)`


## Creating Classes of Private and Public Values

In [127]:
class PublicValue:
    
    def __init__(self, value):
        self.value = value

In [128]:
class PrivateValue:

    def __init__(self, value, share0=None, share1=None, share2=None):
        if not value is None:
            share0, share1, share2 = encrypt(value)
        self.share0 = share0
        self.share1 = share1
        self.share2 = share2

    def decrypt(self):
        return PublicValue(decrypt(self.share0, self.share1, self.share2))
    
    def add(x, y):
        if type(y) is int: y = PublicValue(y)
        if type(y) is PublicValue:
            share0 = (x.share0 + y.element) % Q
            share1 =  x.share1
            share2 =  x.share2
            return PrivateValue(None, share0, share1, share2)
        if type(y) is PrivateValue:
            share0 = (x.share0 + y.share0) % Q
            share1 = (x.share1 + y.share1) % Q
            share2 = (x.share2 + y.share2) % Q
            return PrivateValue(None, share0, share1, share2)

In [129]:
x = PrivateValue(5)
y = PrivateValue(12)

In [134]:
z = x.add(y=y)

In [136]:
decrypt(z.share0, z.share1, z.share2)

17